<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Get-term-score" data-toc-modified-id="Get-term-score-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Get term score</a></span></li></ul></div>

In [1]:
import os
from os import listdir
import pandas as pd
from nltk.corpus import sentiwordnet as swn
df = pd.read_csv('C:/Users/xyyao/Documents/GitHub/Topics-Extraction-Hotel-Reviews - local/hotel-reviews/Datafiniti_Hotel_Reviews_Jun19.csv', header=0)
df.columns

Index(['id', 'dateAdded', 'dateUpdated', 'address', 'categories',
       'primaryCategories', 'city', 'country', 'keys', 'latitude', 'longitude',
       'name', 'postalCode', 'province', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username', 'sourceURLs', 'websites'],
      dtype='object')

In [2]:
df = df[['reviews.text', 'reviews.rating']]

## Get term score

In [3]:
import numpy as np
import re

# nlp libraries/api
import en_core_web_lg
import gensim
from nltk.corpus import sentiwordnet as swn
#from neuralcoref import Coref

spacy = en_core_web_lg.load()
#coref = Coref(nlp=spacy)

In [4]:
import pickle
# Uncomment below if running for first time.
# Setup nltk corpora path and Google Word2Vec location
#google_vec_file = 'GoogleNews-vectors-negative300.bin'
#word2vec = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)
#pickle.dump(word2vec, open("word2vec_google.pkl", 'wb'))

# If above script has been run, load saved word embedding
word2vec = pickle.load(open("word2vec_google.pkl", 'rb'))

In [5]:
def modify_sentiment(token, sentiment):
    for child in token.children:
        # if there's a adj modifier (i.e. very, pretty, etc.) add more weight to sentiment
        # This could be better updated for modifiers that either positively or negatively emphasize
        # can't catch "there are nowhere near enough loungers"
        if ((child.dep_ == "amod") or (child.dep_ == "advmod") or (child.dep_ == "neg")):
            try:
                senti = list(swn.senti_synsets(child, 'r'))[0]
                pos = senti.pos_score()
                neg = senti.neg_score()
                if child.dep_ == "neg":
                    sentiment *= -(neg + 1)
                else:
                    sentiment *= pos + 1
            except:
                continue
    return sentiment


def add_sentiment(token, sent_dict, sentiment):
    if (token.dep_ == "advmod"):
        return sent_dict
    elif (token.dep_ == "acomp"):
        sent_dict = for_acomp(token, sent_dict, sentiment)
    elif (token.dep_ == "amod"):
        sent_dict = for_amod(token, sent_dict, sentiment)
    elif (token.dep_ == "compound"):
        sent_dict = for_compound(token, sent_dict, sentiment)
    elif (token.dep_ == "conj"):
        sent_dict = add_sentiment(token.head, sent_dict, sentiment)
    elif (token.dep_ == "dobj"):
        sent_dict = for_dobj(token, sent_dict, sentiment)
    elif (token.dep_ == "relcl"):
        if token.head.pos_ == "NOUN" or token.head.pos_ == "PROPN":
            sent_dict[token.head.lemma_] += sentiment
    else:
        # if verb, check if there's a direct object
        if token.head.dep_ == "advcl" or token.head.dep_ == "ROOT":
            sent_dict = for_advcl(token.head, sent_dict, sentiment, token)
        for child in token.children:
            if (token.pos_ == "VERB") & (child.dep_ == "dobj") & (child.pos_ != 'PRON'):
                sent_dict = check_compound(child, sent_dict, sentiment, token)
            # check for nouns
            elif child.pos_ == "NOUN" or child.pos_ == "PROPN":
                sent_dict = check_compound(child, sent_dict, sentiment, token)
            if ((token.pos_ == "VERB") or (token.pos_ == "ADJ")) & (child.dep_ == "prep"):
                sent_dict = for_prep(child, sent_dict, sentiment)
    return sent_dict


def add_subchild(child, sent_dict, sentiment, token, added):
    if child.children == []:
        return sent_dict, added
    else:
        conj = 0
        for subchild in child.children:
            if (subchild.dep_ in ["compound", "amod", "nmod"]) & (subchild != token):
                sent_dict[subchild.lemma_ + ' ' + child.lemma_] += sentiment
                for sub in subchild.children:
                    if sub.pos_ in ["NOUN", "PROPN"]:
                        sent_dict[sub.lemma_ + ' ' + child.lemma_] += sentiment
                added = True
            # check for conjugates (a AND b), then add both to dictionary
            elif subchild.text == "and" or subchild.text == ",":
                conj += 1
        for subchild in child.children:
            if (conj > 0) & (subchild.pos_ == "NOUN" or subchild.pos_ == "PROPN"):
                sent_dict = check_compound(subchild, sent_dict, sentiment, child)
                conj -= 1
    return sent_dict, added


def check_compound(child, sent_dict, sentiment, token=None):
    added = False
    sent_dict, added = add_subchild(child, sent_dict, sentiment, token, added)
    if not added:
        sent_dict[child.lemma_] += sentiment
    return sent_dict


def for_compound(token, sent_dict, sentiment):
    if token.head.dep_ == "compound":
        sent_dict = for_compound(token.head, sent_dict, sentiment)
    elif token.head.pos_ in ["NOUN", "PROPN"]:
        sent_dict = check_compound(token.head, sent_dict, sentiment)
    return sent_dict


def for_acomp(token, sent_dict, sentiment):
    for ancestor in token.ancestors:
        if ancestor.dep_ == "attr":
            sent_dict[ancestor] += sentiment
        if ancestor.dep_ in ["advcl", "ROOT", "conj", "ccomp"]:
            sent_dict = for_advcl(ancestor, sent_dict, sentiment, token)
    return sent_dict


def for_advcl(ancestor, sent_dict, sentiment, token):
    for child in ancestor.children:
        if (child.dep_ in ["nsubj", "compound"]) & (child.pos_ in ["NOUN", "PROPN"]):
            sent_dict = check_compound(child, sent_dict, sentiment, token)
    return sent_dict


def for_amod(token, sent_dict, sentiment):
    if token.head.dep_ == "appos":
        sent_dict = check_compound(token.head, sent_dict, sentiment, token)
    elif token.head.dep_ == "pobj":
        if token.head.head.head.dep_ in ["advcl", "ROOT", "conj"]:
            sent_dict = for_advcl(token.head.head.head, sent_dict, sentiment, token)
    if token.head.pos_ == "NOUN" or token.head.pos_ == "PROPN":  # token.head.dep_ != "compound", so "free room delivery" not added twice
        sent_dict = check_compound(token.head, sent_dict, sentiment, token)
    return sent_dict


def for_dobj(token, sent_dict, sentiment):
    if token.head.dep_ == "conj":
        head = token.head.head
        head_child = token.head
    else:
        head = token.head
        head_child = token
    for child in head.children:
        if (child.dep_ == "nsubj") & (child.pos_ == "NOUN" or child.pos_ == "PROPN"):
            sent_dict = check_compound(child, sent_dict, sentiment, head_child)
    return sent_dict


def for_prep(token, sent_dict, sentiment):
    for child in token.children:
        if child.pos_ == "NOUN" or child.pos_ == "PROPN":
            sent_dict = check_compound(child, sent_dict, sentiment, token)
    return sent_dict


def feature_sentiment(sentence, sent_dict):
    '''
    input: dictionary and sentence
    function: appends dictionary with new features if the feature did not exist previously,
              then updates sentiment to each of the new or existing features
    output: updated dictionary
    '''
    sentence = spacy(sentence)
    for token in sentence:
        # check if the word is an opinion word, then assign sentiment
        try:
            senti = list(swn.senti_synsets(token.text, 'a'))[0]
            pos = senti.pos_score()
            neg = senti.neg_score()
            if (pos != 0) or (neg != 0):
                sentiment = pos if pos > neg else -neg
            else:
                continue
            # if target is an adverb modifier (i.e. pretty, highly, etc.)
            # but happens to be an opinion word, ignore and pass
            sentiment = modify_sentiment(token, sentiment)
            sent_dict = add_sentiment(token, sent_dict, sentiment)
        except:
            continue
    #print(sent_dict)
    return sent_dict


from nltk.tokenize import sent_tokenize


def split_sentence(text):
    '''
    splits review into a list of sentences using spacy's sentence parser
    '''
    review = spacy(text)
    bag_sentence = []
    start = 0
    for token in review:
        if token.sent_start:
            bag_sentence.append(review[start:(token.i - 1)])
            start = token.i
        if token.i == len(review) - 1:
            bag_sentence.append(review[start:(token.i + 1)])
    return bag_sentence


# Remove special characters using regex
def remove_special_char(sentence):
    return re.sub(r"[^a-zA-Z0-9.',:;?]+", ' ', sentence)


def check_similarity(aspects, word):
    '''
    checks for word2vec similarity values between category word and the term
    returns most similar word
    '''
    similarity = []
    for aspect in aspects:
        aspect = aspect.lower()
        sim = [word2vec.n_similarity([aspect], word.split())]
        for splited in word.split():
            sim.append(word2vec.n_similarity([aspect], [splited]))
        similarity.append(max(sim))
    # set threshold for max value
    if max(similarity) > 0.2:
        return aspects[np.argmax(similarity)]
    else:
        return None


In [6]:
import pickle
lr_model = pickle.load(open('lr_model.pkl', 'rb'))
mlb = pickle.load(open('mlb.pkl', 'rb'))

In [17]:
from textblob import TextBlob
from collections import Counter

polarities, review_aspect, term_score, aspects = [], [], [], ['hotel']
for x in df['reviews.text']:
    polar, pol_term, n_pol, n_term = 0, 0, 0, 0
    
    try:
        review = x.split('Thank you for your')[0]
        sentences = list(map(str, split_sentence(review)))
        pred = mlb.inverse_transform(lr_model.predict(sentences))
        review_aspect.append(set(list(sum(pred, ()))))
    except:
        review_aspect.append('NOTRELATED')
    try:
        for i, p in enumerate(pred):
            term_dict = Counter()
            if p != ('NOTRELATED',):
                sentence = remove_special_char(sentences[i])
                analysis =TextBlob(sentence)
                polar += analysis.sentiment.polarity
                n_pol += 1
                term_dict = feature_sentiment(sentence.lower(), term_dict)
                for term in term_dict:
                    try:
                        term_aspect = check_similarity(list(p), term)
                        if term_aspect:
                            n_term += 1
                            pol_term += term_dict[term]
                    except:
                        print(term, "not in vocab")
                        continue
        term_score.append(pol_term/n_term)
    except:
        term_score.append(pol_term)
    try:
        polarities.append(polar/n_pol)
    except:
        polarities.append(polar)
        

. location not in vocab
n.o location not in vocab
poydras street not in vocab
poydras not in vocab
york 36hrs not in vocab
new 36hrs not in vocab
days.and not in vocab
90th birthday not in vocab
downtown estes not in vocab
wiu campus not in vocab
-PRON- service not in vocab
squeeze orangejuice not in vocab
fascilitie not in vocab
12th check not in vocab
sheraton not in vocab
sheraton not in vocab
sheraton hotel not in vocab
acknowledgement not in vocab
sheraton not in vocab
gaganjot not in vocab
anacu7065vb not in vocab
mauckfront manager not in vocab
lyle not in vocab
marriot hotel not in vocab
mauckfront manager not in vocab
mrshibariplay not in vocab
mauckfront manager not in vocab
guality not in vocab
fact not in vocab
zacks bar not in vocab
jeannette not in vocab
eatdear guest not in vocab
kind not in vocab
receptionist not in vocab
amentie not in vocab
eyou not in vocab
john k. not in vocab
levine331 not in vocab
airconditon not in vocab
third not in vocab
rightno not in vocab
br

marriott center not in vocab
marriott not in vocab
manager kyley not in vocab
properpty not in vocab
travelocity not in vocab
morewe not in vocab
tralor food not in vocab
breamfast not in vocab
jeramy not in vocab
jxrob not in vocab
aloha furtado not in vocab
pattie furtado not in vocab
aloha wendyb not in vocab
oahu not in vocab
aloha wendyb not in vocab
aloha wendyb not in vocab
aloha wendyb not in vocab
aloha wendyb not in vocab
staff not in vocab
marriott property not in vocab
marriott not in vocab
karris not in vocab
fitness centre not in vocab
blige concert not in vocab
-PRON- not in vocab
tripadvisor review not in vocab
marriott bed not in vocab
horton not in vocab
lodi western not in vocab
neighbour not in vocab
neighbour not in vocab
mcenteefront manager not in vocab
regardsquinn manager not in vocab
comphy bed not in vocab
irma not in vocab
breakfast downstair not in vocab
emmacakes not in vocab
chipottle not in vocab
people not in vocab
pfront manager not in vocab
dustmite a

carone disadvantage not in vocab
.downside not in vocab
400 not in vocab
rooms not in vocab
fairfield inn not in vocab
loosa restaurant not in vocab
bienville house not in vocab
overstuffed bienville not in vocab
chair bienville not in vocab
34 not in vocab
tripadvisor not in vocab
-PRON- not in vocab
bienville not in vocab
bienville house not in vocab
crdonahue not in vocab
bubba gumps not in vocab
stylenice soap not in vocab
bathroomrestaurant bar not in vocab
matterbell staff not in vocab
bienville house not in vocab
waikiki beach not in vocab
baymont requirement not in vocab
croisant not in vocab
breakbast bag not in vocab
-PRON- guy not in vocab
satisfieded statment not in vocab
pacacher not in vocab
. kelly not in vocab
lollye not in vocab
world4 location not in vocab
use not in vocab
odour not in vocab
locationis not in vocab
distnace not in vocab
rosen not in vocab
gym not in vocab
vried breakfast not in vocab
atm.we not in vocab
cheech movie not in vocab
staff not in vocab
la 

williamsgeneral not in vocab
greenbeanjean26,thank not in vocab
-PRON- thirtysomething not in vocab
cruzgeneral not in vocab
bernadette not in vocab
corinne not in vocab
sheraton hotel not in vocab
cecilio l not in vocab
cecilio not in vocab
radisson hotel not in vocab
williamsgeneral not in vocab
size jenga not in vocab
life jenga not in vocab
sfo not in vocab
radisson hotel not in vocab
bathhroom door not in vocab
loews hotel not in vocab
slt not in vocab
southlake not in vocab
bus not in vocab
store not in vocab
posturepedic mattress not in vocab
sealy mattress not in vocab
sealy posturepedic not in vocab
olathe not in vocab
200 not in vocab
camp pendleton not in vocab
camp pendleton not in vocab
epicentre not in vocab
epicentre not in vocab
room verty not in vocab
northville not in vocab
-PRON- not in vocab
n wildvery not in vocab
foodcomfy bed not in vocab
staffnoise not in vocab
fridge not in vocab
n wildvery not in vocab
foodcomfy bed not in vocab
staffnoise not in vocab
fridge 

reynaldo not in vocab
escavation not in vocab
hgi not in vocab
falcontx not in vocab
i35 access not in vocab
honor not in vocab
febreze not in vocab
candace not in vocab
enoch d. not in vocab
alcatraz exhibit not in vocab
stinch not in vocab
100 not in vocab
elavator not in vocab
shower not in vocab
internet downstair not in vocab
hhonors member not in vocab
-PRON- comment not in vocab
. randall not in vocab
. randall not in vocab
. not in vocab
curtious staff not in vocab
brianna not in vocab
darletta hope not in vocab
elite marriott not in vocab
ramh not in vocab
805ronaldw not in vocab
tosca lounge not in vocab
suntrust park not in vocab
double not in vocab
170 not in vocab
best not in vocab
marriott hotel not in vocab
marriott not in vocab
renaissance waverly not in vocab
hotel waverly not in vocab
marriott not in vocab
marriott property not in vocab
amira not in vocab
hotel not in vocab
ucsd not in vocab
mexi food not in vocab
torrey reserve not in vocab
torrey pines not in vocab


western miramar not in vocab
diego miramar not in vocab
san miramar not in vocab
western miramar not in vocab
inn miramar not in vocab
western miramar not in vocab
inn miramar not in vocab
good miramar not in vocab
reasonble price not in vocab
western miramar not in vocab
diego miramar not in vocab
san miramar not in vocab
diegos attraction not in vocab
western miramar not in vocab
san miramar not in vocab
diego miramar not in vocab
western miramar not in vocab
diego miramar not in vocab
san miramar not in vocab
miramar diego not in vocab
miramar diego not in vocab
western miramar not in vocab
san miramar not in vocab
diego miramar not in vocab
terric890i not in vocab
breweries not in vocab
miramar show not in vocab
h20lover not in vocab
western miramar not in vocab
diego miramar not in vocab
san miramar not in vocab
miramar not in vocab
western miramar not in vocab
diego miramar not in vocab
san miramar not in vocab
western miramar not in vocab
san miramar not in vocab
diego miramar n

ugo c not in vocab
tripadvisor review not in vocab
travelingjeffry not in vocab
jenh not in vocab
hyatts not in vocab
sbtimetraveler not in vocab
gareth99,thank not in vocab
tripadvisor review not in vocab
doughnut not in vocab
elevators not in vocab
tripadvisor review not in vocab
qfc not in vocab
breakfast not in vocab
andbexs not in vocab
long wayon not in vocab
positive side1 not in vocab
side1 not in vocab
tripadvisor review not in vocab
susandy7842gy not in vocab
tripadvisor review not in vocab
jenh not in vocab
brekke not in vocab
restaurant not in vocab
beerdamage not in vocab
complaint not in vocab
hotel not in vocab
johntraveler1980,thank not in vocab
susandy7842gy not in vocab
tripadvisor review not in vocab
dear 373jkim not in vocab
seatno poolrooftop not in vocab
complaint not in vocab
hotel not in vocab
kaylangerardw not in vocab
tripadvisor review not in vocab
chana100,we not in vocab
dear 444diane not in vocab
perfect 444diane not in vocab
kaylangerardw not in vocab
tri

cebkilifi not in vocab
ingrid not in vocab
virgirl808 not in vocab
30 not in vocab
. not in vocab
centre holiday not in vocab
gshaw not in vocab
mattres not in vocab
. hotel not in vocab
neighbourhood not in vocab
. charles not in vocab
michaelww9944qs not in vocab
. lamp not in vocab
. therese not in vocab
st therese not in vocab
. not in vocab
65 not in vocab
270.00 not in vocab
batteriesoverall not in vocab
union . not in vocab
20frederico1st not in vocab
proofinghi not in vocab
12th room not in vocab
longtime marriott not in vocab
marriott traveler not in vocab
80 not in vocab
running not in vocab
francesca suh not in vocab
francesca suh not in vocab
faris not in vocab
nagaandkambing not in vocab
usna70 not in vocab
francesca suh not in vocab
pengembaraabadi not in vocab
francesca suh not in vocab
marriott not in vocab
marriott member not in vocab
hello jbsdtraveler not in vocab
faris not in vocab
francesca suh not in vocab
size not in vocab
. marglowe3 not in vocab
mr marglowe3 no

In [18]:
len(polarities), len(review_aspect), len(term_score)

(10000, 10000, 10000)

In [19]:
df['polarity']=polarities
df['term_score'] = term_score
df['aspects'] = review_aspect

In [20]:
pd.set_option('display.max_colwidth', -1)
df

C:\Users\xyyao\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,reviews.text,reviews.rating,polarity,term_score,aspects
0,"This hotel was nice and quiet. Did not know, there was train track near by. But it was only few train passed during our stay. Best Western changed hotel classification. The Plus category are not the same as before.",3,0.233333,0.625000,"{BUILDING, OTHER, NOTRELATED}"
1,"We stayed in the king suite with the separation between the bedroom and the living space. The sofa bed wasn't very good I had back discomfort by the day we left on our three night stay. The room is clean, and the king bed very comfortable. This hotel is located within walking distance to most places you will want to... More",4,0.273889,0.343750,"{LOCATION, OTHER, CLEANLINESS, ROOMS}"
2,"Parking was horrible, somebody ran into my rental car while staying there. I didn't get to try the breakfast, I was there for business so the restaurant opened to late for the business world to enjoy, I had to asked for coffee for my room, And the items in the vending machine were stale.",3,-0.450000,-0.375000,"{FOOD, BUSINESS, OTHER, ROOMS}"
3,"Not cheap but excellent location. Price is somewhat standard for not hacing reservations. But room was nice and clean. They offer good continental breakfast which is a plus and compensates. Front desk service and personnel where excellent. It is Carmel, no A/C in rooms but they have a fan for air circulation.",5,0.430556,0.500000,"{FOOD, CLEANLINESS, SERVICE, VALUE, LOCATION, ROOMS}"
4,"If you get the room that they advertised on the website and for what you paid, you may be lucky.If you stay many days , they will give you the not so good rooms.Nobody wants to stay in these rooms: low light/dark rooms, near pool, noisy, smelly bathrooms, or difficult access. If you stay one-two days you will get probably... More",2,0.288333,-0.875000,"{VALUE, OTHER, ROOMS}"
...,...,...,...,...,...
9995,My friends and I took a trip to Hampton for the Basslights show and booked us a couple rooms for Saturday and Sunday. When we arrived around 7am we realized we should have booked more than 2 rooms because of how many people we had and went to the concierge to check if there were any additional rooms available. They... More,4,0.322222,0.000000,"{OTHER, CHECKIN, NOTRELATED}"
9996,"from check in to departure, staff is friendly, professional and wanting to help. the property is clean, the rooms clean and quiet and very comfortable. the area made me feel very safe. i even walked to my work and other businesses. parking is very adequate.",5,0.301833,-0.458333,"{CLEANLINESS, SERVICE, VALUE, LOCATION, ROOMS}"
9997,"This Hampton is located on a quiet street across from a hospital, located minutes from Langley/Ft Eutis Base. Easy access to I-40. A hop skip and jump from Virginia Beach, and Williamsburg. Overall the hotel was busy with lots of guests but not overcrowded. The rooms were clean and comfortable.",5,0.116667,-0.437500,"{CLEANLINESS, NOTRELATED, OTHER, LOCATION, ROOMS}"
9998,"Awesome wings (my favorite was garlic parmesan) and a great atmosphere! Not to mention they have an awesome music selection, a lottery machine and arcade games to cure your boredom! The owners are amazing!Thank you!",5,0.788889,0.875000,"{FOOD, SERVICE, OTHER}"


In [21]:
from sklearn.model_selection import train_test_split

x = df[['polarity', 'term_score']]
y = df['reviews.rating']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [22]:
from sklearn.linear_model import LinearRegression
lrModel = LinearRegression()

#训练模型
lrModel.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [23]:
y_pred = lrModel.predict(X_test)

In [24]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

0.8627087052484326

In [25]:
filename = 'linear_model.pkl'
pickle.dump(lrModel, open(filename, 'wb'))

In [36]:
df_pred = df.iloc[X_test.index]
df_pred['pred_rating'] = y_pred
df_pred = df_pred[['reviews.text','aspects','reviews.rating','pred_rating', 'polarity', 'term_score']]
df_pred

C:\Users\xyyao\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,reviews.text,aspects,reviews.rating,pred_rating,polarity,term_score
2738,"Close to downtown and Rocky Mountain National Park. You can go out back and fish for trout. Wonderful views. Very clean and quiet! Fell asleep to the sound of the river, very relaxing. Great hosts. Great location! We'll be back!","{FOOD, CLEANLINESS, OTHER, LOCATION, ROOMS}",5,4.597421,0.392292,-0.125000
738,"Rooms are big. All the amenities from bottled water, single coffee maker and the pods, toiletries, good wifi, full cable, ironing board/iron, mini fridge , comfortable beds and very accommodating staff, it's hard not to give it a good rating. Considering construction on some floors were ongoing, the management has been good at communicating details on a daily basis the... More","{NOTRELATED, OTHER, ROOMS}",5,4.101097,0.261281,0.000000
7447,We were walking along the beach and were hungry. Nice lunch right beside the pool. Grilled cheese was amazing and the Caprese salad is quite good as well. The staff was extremely friendly and helpful.,"{BUILDING, SERVICE, OTHER}",4,4.616268,0.366964,0.500000
7564,"Keith and Denise are friendly and delightful. Thier BB is located in a lovely peaceful location and is sparking clean. Breakfast was gourmet and served on the beautiful patio. You will love Zorro, the black standard poodle","{FOOD, SERVICE, LOCATION, OTHER}",5,5.153255,0.505208,0.437500
9828,"Hotel location is great. Yards from Michigan but without the chaos. You are greeted by a valet with a smile who asks that you check in on the 20th floor. More smiling, happy folks at the reception desk. Our room was remarkable. The view was terrific but the room was tastefully appointed (a bit on the modern side). We had breakfast at the hotel daily and was very good. The complimentary juice of the day was kinda cool. Roof top bar was hopping and comfortable. All in all, young very accommodating staff, great location and service. We have stayed in Chicago many times. Our new go to place.Dear Sunburned 31, Thank you for taking the time to write a review of your stay. I'm so glad you enjoyed your visit and all of our amenities and services. The team tries hard to make each and every guest happy and I'm glad we were able to turn you into a loyal guest. We look forward to your swift return. Kind Regards, Tod Chambers","{FOOD, SERVICE, NOTRELATED, OTHER, BUILDING, CHECKIN, LOCATION, ROOMS}",5,4.356646,0.335091,-0.196429
...,...,...,...,...,...,...
1460,"We had a great stay at the Sorrento. Staff was excellent very helpful with suggestions and making reservations for meals, etc. . We stayed in a Queen Suite. It's and older hotel with only 76 rooms but that's the type of hotel we like to stay in. They have a car that will take you anywhere in the downtown area. Location is excellent. Very easy to get around. There's a bar restaurant plus more restaurants and bars within walking distance. we love this review! Thank you for such kind words. Appreciate the praise!Randall ObrechtGeneral ManagerSorrento Hotel","{SERVICE, LOCATION, OTHER, NOTRELATED}",5,4.486758,0.328333,0.625000
3164,"I stayed with a friend at the Freehand for two nights in a queen bed private room. The room was quiet and clean, but very small. My friend and I were climbing over each other and our suitcases to move around the room. One thing that ruined the room for us was the lack of control over the temperature of the room. The weather in Chicago that week was in the mid-upper 60s, but the room was WARM. There's an A/C unit in the room, but no matter what we did to fiddle with it, it just would not get cool. Tiny room + humidity from showers + stubborn A/C + poor window ventilation stuffy stuffy stuffy.What also wasn't fun was being charged an extra 6.35 on my credit card when I checked my billing statement a week after checking out. I called in to ask about the charge and was told we had used the water in the mini bar. We had not. The front desk employee was nice enough to remove the charge, but is it not s